In [11]:
# 모델 저장과 복원

# 설정
## 설치와 import
import os
import sys
from google.colab import drive

# 드라이브 마운트 및 패키지 경로 설정
# drive.mount("/content/drive", force_remount=True)
# my_path = "/content/package"
# save_path = "/content/drive/MyDrive/Colab Notebooks/package"

# os.symlink(save_path, my_path)
# sys.path.insert(0, my_path)

# Required to save models in HDF5 format
# !pip install --target=$my_path pyyaml h5py

import tensorflow as tf
from tensorflow.keras import layers

## 예제(MNIST) 데이터셋 받기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

## 모델 정의
# Define a simple sequential model
def create_model():
    model = tf.keras.Sequential([
        layers.Dense(512, activation='relu', input_shape=(784,)),
        layers.Dropout(0.2),
        layers.Dense(10)])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
    return model

# Create a basic model instance
# model = create_model()

# Display the model's architecture
# model.summary()

# 훈련 중 체크포인트 저장 (tf.keras.callbacks.ModelCheckpoint 콜백 사용)
## 체크포인트 콜백 사용
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

# Train the model with the new callback
# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.
# model.fit(train_images, train_labels, epochs=10,
#           validation_data=(test_images, test_labels),
#           callbacks=[cp_callback])                      # Pass callback to training


# os.listdir(checkpoint_dir)

# Create a basic model instance
# model = create_model()

# Evaluate the model
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

# Load the weights
# model.load_weights(checkpoint_path)

# Re-evaluate the model
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## 체크포인트 콜백 매개변수
# Include the epoch in the file name (uses 'str.format')
# checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)

# batch_size = 32

# # Create a callback that saves the model's weights every 5 epoch
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  verbose=1,
#                                                  save_weights_only=True,
#                                                  save_freq=5 * batch_size)

# # Create a new model instance
# model = create_model()

# # Save the weights using the 'checkpoint_path' format
# model.save_weights(checkpoint_path.format(epoch=0))

# # Train the model with the new callback
# model.fit(train_images, train_labels,
#           epochs=50, batch_size=batch_size,
#           callbacks=[cp_callback],
#           validation_data=(test_images, test_labels), verbose=0)

# # os.listdir(checkpoint_dir)

# latest = tf.train.latest_checkpoint(checkpoint_dir)
# # latest

# # Create a new model instance
# model = create_model()

# # Load the previously saved weights
# model.load_weights(latest)

# # Re-evaluate the model
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## 파일 정보
# 이진 형식의 훈련된 가중치만 포함하는 체크포인트 형식의 파일 모음에 가중치 저장
# 체크포인트에는 모델의 가중치를 포함하는 하나 이상의 샤드와
# 어떤 가중치가 어떤 샤드에 저장되어 있는지 나타내는 인덱스 파일이 포함
# 단일 머신에서 모델을 훈련하는 경우 접미사가 .data-00000-of-00001인 하나의 샤드를 가짐.

# 수동으로 가중치 저장 (tf.keras.Model.save_weights를 사용)
# 기본적으로 tf.keras, 그리고 특히 Model.save_weights 메서드는 .ckpt 확장자가 있는 Tensorflow 체크포인트 형식 사용
# Save the weights
# model.save_weights("./checkpoints/my_checkpoint")

# # Create a new model instance
# model = create_model()

# # Restore the weights
# model.load_weights("./checkpoints/my_checkpoint")

# # Evaluate the model
# loss, acc = model.evaluate(test_images, test_labels, verbose=2)
# print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

# 전체 모델 저장 (tf.keras.Model.save 호출)
## SavedModel 포맷
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel
!mkdir -p saved_model
model.save("saved_model/my_model")

# my_model directory
!ls saved_model

# Contains an asset folder, saved_model.pb, and variables folder
!ls saved_model/my_model

new_model = tf.keras.models.load_model("saved_model/my_model")

# Check its architecture
new_model.summary()

# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))
print(new_model.predict(test_images).shape)

## HDF5 파일로 저장
# Create and train a new model instance
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file
# The '.h5' extension indicates that the model should be saved to HDF5
model.save('my_model.h5')

# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## 사용자 정의 객체
# HDF5 : 객체 구성을 사용하여 모델 구조를 저장
# SavedModel : 실행 그래프 저장
# -> SavedModel은 원본 코드 없이도 서브클래싱된 모델 및 사용자 지정 레이어와 같은 사용자 지정 객체 저장 가능



Epoch 5: saving model to training_2/cp-0005.ckpt

Epoch 10: saving model to training_2/cp-0010.ckpt

Epoch 15: saving model to training_2/cp-0015.ckpt

Epoch 20: saving model to training_2/cp-0020.ckpt

Epoch 25: saving model to training_2/cp-0025.ckpt

Epoch 30: saving model to training_2/cp-0030.ckpt

Epoch 35: saving model to training_2/cp-0035.ckpt

Epoch 40: saving model to training_2/cp-0040.ckpt

Epoch 45: saving model to training_2/cp-0045.ckpt

Epoch 50: saving model to training_2/cp-0050.ckpt
32/32 - 0s - loss: 0.4877 - sparse_categorical_accuracy: 0.8750 - 210ms/epoch - 7ms/step
Restored model, accuracy: 87.50%
